In [1]:
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

import cv2
import numpy as np
import open3d as o3d
from PIL import Image
import OpenEXR
import array
import Imath
from inference import Inferencer


def draw_point_cloud(color, depth, camera_intrinsics, use_mask = False, use_inpainting = False, scale = 1000.0, inpainting_radius = 5, fault_depth_limit = 0.2, epsilon = 0.01):
    """
    Given the depth image, return the point cloud in open3d format.
    The code is adapted from [graspnet.py] in the [graspnetAPI] repository.
    """
    d = depth.copy()
    c = color.copy() / 255.0
    
    if use_inpainting:
        fault_mask = (d < fault_depth_limit * scale)
        d[fault_mask] = 0
        inpainting_mask = (np.abs(d) < epsilon * scale).astype(np.uint8)  
        d = cv2.inpaint(d, inpainting_mask, inpainting_radius, cv2.INPAINT_NS)

    fx, fy = camera_intrinsics[0, 0], camera_intrinsics[1, 1]
    cx, cy = camera_intrinsics[0, 2], camera_intrinsics[1, 2]

    xmap, ymap = np.arange(d.shape[1]), np.arange(d.shape[0])
    xmap, ymap = np.meshgrid(xmap, ymap)

    points_z = d / scale
    points_x = (xmap - cx) / fx * points_z
    points_y = (ymap - cy) / fy * points_z
    points = np.stack([points_x, points_y, points_z], axis = -1)

    if use_mask:
        mask = (points_z > 0)
        points = points[mask]
        c = c[mask]
    else:
        points = points.reshape((-1, 3))
        c = c.reshape((-1, 3))
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(points)
    cloud.colors = o3d.utility.Vector3dVector(c)
    return cloud


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
sample_root = 'data/scene21/2'

rgb = np.array(Image.open(sample_root + '/rgb1.png'), dtype = np.float32)
depth = np.array(Image.open(sample_root + '/depth1.png'), dtype = np.float32)
depth_gt = np.array(Image.open(sample_root + '/depth1-gt.png'), dtype = np.float32)

print(depth.shape)

depth = depth / 1000
depth_gt = depth_gt / 1000

(720, 1280)


In [2]:
depth = cv2.imread('000000080-transparent-depth-img.exr',  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)[:, :, 0]
depth_gt = cv2.imread('000000080-opaque-depth-img.exr',  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)[:, :, 0] 
rgb = np.array(Image.open('000000080-transparent-rgb-img.jpg'))

In [10]:
depth = cv2.imread('scene12_IMG_DepthMap.tif',  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
depth_gt = cv2.imread('scene12-gt_IMG_DepthMap.tif',  cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
rgb = np.array(Image.open('secondcamera.jpg'))
#rgb = np.array([rgb, rgb, rgb])
#rgb = np.transpose(rgb, (1, 2, 0))

depth = depth / 1000
depth_gt = depth_gt / 1000

In [11]:
print(depth.shape)

print(np.amax(depth))
#cv2.imshow('', rgb)
#cv2.waitKey(0)

(1200, 1680)
1.6723491


In [3]:
inferencer = Inferencer()
res, depth_n = inferencer.inference(np.copy(rgb), np.copy(depth), depth_coefficient = 3, inpainting = True)
#res, depth_n = inferencer.inference(np.copy(rgb), np.copy(depth), depth_coefficient = 10, inpainting = True, target_size = (1680, 1200))

cam_intrinsics = np.load('camIntrinsics-PhoXi.npy')
#cam_intrinsics = np.load('camIntrinsics-clear.npy')

#res = np.clip(res, 0.3, 1.0)
#depth = np.clip(depth, 0.3, 1.0)

In [5]:
cloud_raw = draw_point_cloud(rgb, depth_n, cam_intrinsics, scale = 0.2)
cloud = draw_point_cloud(rgb, res, cam_intrinsics, scale = 0.2)
cloud_gt = draw_point_cloud(rgb, depth_gt, cam_intrinsics, scale = 0.2)

frame = o3d.geometry.TriangleMesh.create_coordinate_frame(0.1)
sphere = o3d.geometry.TriangleMesh.create_sphere(0.002,20).translate([0,0,0.490])
o3d.visualization.draw_geometries([cloud])

In [7]:
vis = o3d.visualization.VisualizerWithKeyCallback()
vis.create_window()

vis.add_geometry(cloud_raw)
#vis.add_geometry(cloud)

def key_callback(key):
    #vis.remove_geometry(cloud_raw)
    vis.add_geometry(cloud)
    vis.update_renderer()

#render_option = vis.get_render_option()
#render_option.point_size = 0.5

vis.register_key_callback(ord('1'), key_callback)

vis.run()
vis.destroy_window()